In [ ]:
!pip install kagglehub[pandas-datasets]
!pip install -U transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = 'philomath-1209/programming-language-identification'
loaded_tokenizer = AutoTokenizer.from_pretrained(model_name)
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
def classify_code(codes, model, tokenizer, device=device):
    # Filter out empty or invalid codes
    codes = [c if isinstance(c, str) and c.strip() else " " for c in codes]

    # Tokenize batch
    inputs = tokenizer(
        codes,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass on GPU
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1).tolist()
    labels = [model.config.id2label[i] for i in predicted_ids]
    return labels


texts = [
    """public class HelloWorld { public static void Main(String[] args) { System.out.println("Hello World!"); } }""",
    """print("hello world")"""
]

labels = classify_code(texts, loaded_model, loaded_tokenizer)
print(labels)

['Java', 'Lua']


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from pandas import DataFrame
from tqdm import tqdm
from numpy import std
import re

In [ ]:
def load_dataset(file_path):
  df: DataFrame  = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "wilfriedkonan/cod-blocks",
    file_path,
  )
  return df

datasets = {
    "c": "c_cleaned.json",
    "csharp": "csharp.json",
    "java": "java_cleaned.json",
    "javascript": "javascript_cleaned.json",
    "php": "php.json",
    "python": "python_cleaned.json",
    "other": "unknown.json"
}

In [ ]:

df = load_dataset(datasets["other"])
BATCH_SIZE = 8

for start_idx in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing batches"):
    end_idx = min(start_idx + BATCH_SIZE, len(df))
    batch_codes = df.loc[start_idx:end_idx, "code"].tolist()

    # Skip empty code
    non_empty_indices = [i for i, c in enumerate(batch_codes) if isinstance(c, str) and c.strip()]
    if not non_empty_indices:
        continue

    filtered_codes = [batch_codes[i] for i in non_empty_indices]

    # Predict languages
    batch_predictions = classify_code(filtered_codes, loaded_model, loaded_tokenizer)

    # Update dataframe only for non-empty entries
    for idx, pred in zip(non_empty_indices, batch_predictions):
        df.at[start_idx + idx, "language"] = pred

# Save the cleaned dataset
df.to_json("other_cleaned.json", orient="records", lines=True)

/tmp/ipython-input-2394601154.py:2: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df: DataFrame  = kagglehub.load_dataset(
Processing batches: 100%|██████████| 30683/30683 [35:38<00:00, 14.35it/s]
